Let's start by installing the Python Libraries we neeed

In [8]:
!pip install -U llama-stack-client dotenv


When running this code in a regular Python application, we would usually like to read environment variables from an `.env` file, for our needs in this lab, we will hard code these in this cell, to make things more clear

In [9]:
# Load environment variables from .env file
from dotenv import load_dotenv
load_dotenv()

# for our lab, we will just define our variables manualy here:
os.environ['LLAMA_STACK_SERVER'] = 'http://localhost:8321'
os.environ['LLAMA_STACK_MODEL'] = 'meta-llama/Llama-3.2-3B-Instruct'

False

As a first step, let's define our client, provide it our Llama-Stack Server location and select the model we would like to work with, later, we will see that pointing this to a different location (Llama-Stack Serve) is all we would need to do to move to a production environment.

In [10]:
import os
from llama_stack_client import LlamaStackClient

LLAMA_STACK_SERVER=os.getenv("LLAMA_STACK_SERVER")
LLAMA_STACK_MODEL=os.getenv("LLAMA_STACK_MODEL")

client = LlamaStackClient(base_url=LLAMA_STACK_SERVER)

# List available models
models = client.models.list()
print("--- Available models: ---")
for m in models:
    print(f"{m.identifier} - {m.provider_id} - {m.provider_resource_id}")


--- Available models: ---
all-MiniLM-L6-v2 - ollama - all-minilm:latest
meta-llama/Llama-3.2-3B-Instruct - ollama - llama3.2:3b-instruct-fp16


Now that our client is set up, let's go through some very simple code snippets, to get you familiar with the syntex. If you used other AI Frameworks, this will soon feel very familiar, as Llamastack follows similar principals and terminology, while allowing a standard to help you quickly shift different components in and out 

In [27]:
response = client.inference.chat_completion(
    model_id=LLAMA_STACK_MODEL,
    messages=[
        {"role": "system", "content": "You're a helpful assistant."},
        {
            "role": "user",
            "content": "What is the top speed of a leopard?",
        },
    ],
    # temperature=0.0, 
)
print(response.completion_message.content)

The top speed of a leopard can vary depending on several factors, such as the individual animal's age, sex, and motivation (e.g., chasing prey or escaping from predators). However, according to various sources, including the National Geographic and the World Wildlife Fund, the average running speed of a leopard is around 40-50 km/h (25-31 mph).

However, some leopards have been clocked at speeds of up to 60 km/h (37 mph) over short distances, such as when chasing prey or escaping from danger. In fact, one study found that a male leopard can reach speeds of up to 70 km/h (43.5 mph) for brief periods.

It's worth noting that leopards are agile and powerful climbers, and they often use their speed and agility to navigate through dense forests and rocky terrain with ease.


Often, we want the LLM to provide us a specific answer, not in a conversational manner, using structured data can be helpful, later in the lab, you will see how we want certain agents to give us specific facts and not a short story about the facts.

Try different animals, to see how the structured data can be helpful for us:

In [28]:
from pydantic import BaseModel
import json

class AnimalSpeed(BaseModel):
    speed: int
    animal: str
    metric_type: str

response = client.inference.chat_completion(
    model_id=LLAMA_STACK_MODEL,
    messages=[
        {"role": "system", "content": "You're a helpful assistant."},
        {
            "role": "user",
            "content": "What is the top speed of a leopard?",            
        },
    ],
    stream=False,    
    response_format={
            "type": "json_schema",
            "json_schema": AnimalSpeed.model_json_schema(),
        }
)


try:
    response_data = json.loads(response.completion_message.content)
    animal = AnimalSpeed(**response_data)    
    print("-------")
    print("Speed: ", animal.speed)
    print("Animal: ", animal.animal)
    print("metric_type: ", animal.metric_type)
    print("-------")
except (json.JSONDecodeError, ValueError) as e:
    print(f"Invalid format: {e}")


-------
Speed:  80
Animal:  leopard
metric_type:  km/h
-------
